In [ ]:
#export
from fastai2.basics import *
from fastai2.tabular.core import *
from fastai2.tabular.model import *

In [ ]:
from nbdev.showdoc import *

In [ ]:
#default_exp tabular.learner

# Tabular learner

> The function to immediately get a `Learner` ready to train for tabular data

## Main functions

In [ ]:
#export
class TabularLearner(Learner):
    "`Learner` for tabular data"
    def predict(self, row):
        tst_to = self.dls.valid_ds.new(pd.DataFrame(row).T)
        tst_to.process()
        dl = self.dls.valid.new(tst_to)
        inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
        i = getattr(self.dls, 'n_inp', -1)
        b = (*tuplify(inp),*tuplify(dec_preds))
        full_dec = self.dls.decode((*tuplify(inp),*tuplify(dec_preds)))
        return full_dec,dec_preds[0],preds[0]

In [ ]:
#export
@delegates(Learner.__init__)
def tabular_learner(dls, layers=None, emb_szs=None, config=None, n_out=None, y_range=None, 
                    ps=None, embed_p=0., use_bn=True, bn_final=False, **kwargs):
    "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
    if config is None: config = tabular_config()
    if layers is None: layers = [200,100]
    to = dls.train_ds
    emb_szs = get_emb_sz(dls.train_ds, {} if emb_szs is None else emb_szs)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be infered from data, set `dls.c` or pass `n_out`"
    model = TabularModel(emb_szs, len(dls.cont_names), n_out, layers, y_range=y_range,
                         ps=ps, embed_p=embed_p, use_bn=use_bn, bn_final=bn_final, **config)
    return TabularLearner(dls, model, **kwargs)

In [ ]:
#export
@typedispatch
def show_results(x:Tabular, y:Tabular, samples, outs, ctxs=None, max_n=10, **kwargs):
    df = x.all_cols[:max_n]
    for n in x.y_names: df[n+'_pred'] = y[n][:max_n].values
    display_df(df)

## Integration example with training

In [ ]:
from fastai2.callback.all import *

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df))

In [ ]:
to = TabularPandas(df, procs, cat_names, cont_names, y_names="salary", splits=splits)

In [ ]:
dls = to.dataloaders(bs=64)
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num,salary
0,5.0,5.0,3.0,9.0,1.0,5.0,1.0,1.0,1.0,0.763051,-0.642616,-2.779907,0.0
1,5.0,6.0,3.0,15.0,1.0,5.0,1.0,1.0,1.0,0.395978,-0.151944,-2.387837,1.0
2,6.0,15.0,3.0,15.0,1.0,5.0,1.0,1.0,1.0,1.350368,-0.163628,1.924936,1.0
3,1.0,7.0,5.0,1.0,5.0,5.0,1.0,1.0,1.0,1.717441,0.124893,-1.995767,0.0
4,7.0,16.0,6.0,13.0,2.0,5.0,1.0,1.0,1.0,1.350368,-0.526912,-0.035416,0.0
5,5.0,1.0,6.0,8.0,2.0,3.0,1.0,1.0,1.0,1.790856,-0.134058,-1.603697,0.0
6,8.0,12.0,3.0,12.0,1.0,1.0,1.0,1.0,1.0,1.570612,-1.542389,-0.427486,1.0
7,5.0,10.0,5.0,5.0,3.0,5.0,1.0,1.0,1.0,-0.338168,0.776385,1.140795,0.0
8,7.0,7.0,3.0,4.0,1.0,5.0,1.0,1.0,1.0,-0.044510,0.522225,-1.995767,0.0
9,7.0,13.0,1.0,13.0,2.0,5.0,1.0,1.0,1.0,1.130124,-0.247319,1.532865,0.0


In [ ]:
type(dls.train_ds)

fastai2.tabular.core.TabularPandas

In [ ]:
dls.train_ds.classes

AttributeError: classes

In [ ]:
learn = tabular_learner(dls, [200,100], metrics=accuracy)

AttributeError: classes

In [ ]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.361968,0.359497,0.834152,00:11


In [ ]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,Private,Some-college,Never-married,Other-service,Own-child,Black,False,False,False,21.000000,315303.006246,10.0,<50k,<50k
1,Private,HS-grad,Never-married,Adm-clerical,Unmarried,Black,False,False,False,22.000000,310380.006155,9.0,<50k,<50k
2,Local-gov,Masters,Married-civ-spouse,Prof-specialty,Wife,White,False,False,False,50.000000,259377.002261,14.0,>=50k,>=50k
3,Private,Bachelors,Never-married,Prof-specialty,Own-child,White,False,False,False,27.000000,289146.998294,13.0,<50k,<50k
4,Private,11th,Never-married,Priv-house-serv,Own-child,White,False,False,False,16.999999,295308.002387,7.0,<50k,<50k
5,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,False,False,33.000000,170769.000613,9.0,<50k,<50k
6,State-gov,HS-grad,Never-married,Other-service,Not-in-family,White,False,False,False,41.000000,210093.999594,9.0,<50k,<50k
7,Private,HS-grad,Married-spouse-absent,Other-service,Not-in-family,Asian-Pac-Islander,False,False,False,33.000000,122612.000014,9.0,<50k,<50k
8,?,Bachelors,Married-civ-spouse,?,Husband,White,False,False,False,34.000000,35594.997420,13.0,<50k,<50k
9,Private,11th,Never-married,Machine-op-inspct,Not-in-family,White,False,False,False,19.000000,99246.002236,7.0,<50k,<50k


In [ ]:
learn.predict(df.iloc[0])

(  workclass    education       marital-status occupation relationship    race  \
 0   Private   Assoc-acdm   Married-civ-spouse       #na#         Wife   White   
 
   age_na fnlwgt_na education-num_na   age         fnlwgt  education-num salary  
 0  False     False            False  49.0  101319.999822           12.0   <50k  ,
 tensor(0),
 tensor([0.5456, 0.4544]))

In [ ]:
#TODO: Fix prod for tabular
#learn.export()
#learn1 = torch.load('export.pkl')
#learn1.predict(df.iloc[0])

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_torch_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.external.ipynb.
Converted 05_data.transforms.ipynb.
Converted 06_data.block.ipynb.
Converted 07_vision.core.ipynb.
Converted 08_vision.data.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 09c_vision.widgets.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 19_callback.mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Converted 21_vision.learner.ipynb.
Converted 22_tutorial.imagenette.ipynb.
Converted 23_tutoria